In [ ]:
# INSTRUCCIÓN: Ejecutar esta celda primero para cargar todas las librerías necesarias

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import random
from itertools import product
import warnings
warnings.filterwarnings('ignore')

# Fijar semillas para reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("✓ Librerías importadas correctamente")
print(f"TensorFlow version: {tf.__version__}")

In [ ]:
#INSTRUCCIÓN: Ejecutar para cargar el CSV y hacer preprocesamiento básico

df = pd.read_csv("registros_rio_6746.csv")

# Modificar columnas del df
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(by="date")


# Agregar columnas futuras (las vamos a eliminar después)
df['altura_7_dias'] = df['altura_value'].shift(periods=-7)
df['precipitaciones_7_dias'] = df['precipitaciones_value'].shift(periods=-7)
df = df.iloc[:1001].copy()  # elimino las ultimas 7 ya que tienen datos del futuro

print(f"✓ Datos cargados: {len(df)} registros")
print(f"  Rango de fechas: {df['date'].min()} a {df['date'].max()}")

In [ ]:
# INSTRUCCIÓN: Ejecutar para agregar la columna de estación del año con One-Hot Encoding

import pandas as pd

def obtener_estacion(fecha):
    mes = fecha.month
    dia = fecha.day
    
    if (mes == 12 and dia >= 21) or (mes <= 3 and (mes < 3 or dia <= 20)):
        return "verano"
    elif (mes == 3 and dia >= 21) or (mes <= 6 and (mes < 6 or dia <= 20)):
        return "otoño"
    elif (mes == 6 and dia >= 21) or (mes <= 9 and (mes < 9 or dia <= 20)):
        return "invierno"
    else:
        return "primavera"

# Crear la columna base
df['estacion'] = df['date'].apply(obtener_estacion)

# Aplicar One-Hot Encoding
df = pd.get_dummies(df, columns=['estacion'], prefix='est')

print("✓ Columnas de estación creadas con One-Hot Encoding\n")
print("Nuevas columnas:")
print([col for col in df.columns if col.startswith("est_")])



In [ ]:
# INSTRUCCIÓN: Ejecutar para eliminar columnas que no usaremos

columnas_a_eliminar = ['Unnamed: 0', 'rio_id', 'lat', 'lon', 'altura_7_dias', 'precipitaciones_7_dias']
df = df.drop(columns=columnas_a_eliminar, errors='ignore')

print(f"✓ Columnas limpiadas")
print(f"  Columnas finales: {list(df.columns)}") 


In [ ]:
# INSTRUCCIÓN: CRÍTICO - Esta celda separa los datos ANTES de hacer K-Fold
# El test set NO se tocará hasta el final

# Primero separamos el TEST SET (20% final del dataset)
# Este conjunto NO se usará para nada hasta la evaluación final
test_size = 0.20
split_point = int(len(df) * (1 - test_size))

df_train_val = df.iloc[:split_point].copy()  # 80% para train+validation
df_test = df.iloc[split_point:].copy()       # 20% para test final

print("=" * 80)
print("SEPARACIÓN DE DATOS")
print("=" * 80)
print(f"Dataset completo: {len(df)} registros")
print(f"Train+Validation: {len(df_train_val)} registros (80%)")
print(f"Test (guardado para el final): {len(df_test)} registros (20%)")
print("\n⚠️ IMPORTANTE: El test set NO se usará hasta la evaluación final")
print("=" * 80)


In [ ]:
# INSTRUCCIÓN: Ejecutar para definir la función de ventanas deslizantes (ajustada a nueva convención)
import numpy as np

# features debe ser: ['altura_value', 'precipitaciones_value', 'est_verano', 'est_otoño', 'est_invierno', 'est_primavera']
def crear_ventanas(df, features, window_size=3):
    """
    Nuevo comportamiento:
    - window_size = número de días pasados a observar (p.ej. 2 -> i-2, i-1).
    - la historia incluye además el día 'hoy' (i). Por tanto la historia tiene window_size+1 días: i-window_size .. i.
    - se añaden como features del "futuro" las columnas features[1:] (precip + estaciones) en i+1.
    - target = altura_value en i+1.

    Esto produce muestras que predicen la altura de mañana (i+1) usando pasado + presente + precip/estación de mañana.
    """
    X = []
    y = []

    hist_cols = features              # columnas que usamos por día en la historia (altura, precip, estaciones)
    future_cols = features[1:]        # para el día objetivo solo usamos precip + estaciones

    # iteramos t = índice «hoy»; necesitamos que exista t+1 para target
    for t in range(window_size, len(df) - 1):
        # historia: desde t-window_size hasta t (inclusive) -> window_size+1 días
        hist_block = df[hist_cols].iloc[t - window_size : t + 1].values.flatten()
        # futuro: precip + estación en t+1
        fut_block = df[future_cols].iloc[t + 1].values.flatten()

        features_vector = np.concatenate([hist_block, fut_block])
        X.append(features_vector)
        # target: altura en t+1
        y.append(df['altura_value'].iloc[t + 1])

    return np.array(X), np.array(y)

print('✓ crear_ventanas: nueva convención aplicada (history: i-window_size..i, futuro: i+1, target: altura i+1)')

In [ ]:
# INSTRUCCIÓN: Ejecutar para definir el K-Fold temporal (respeta el orden cronológico)

class TimeSeriesKFold:
    """
    K-Fold Cross-Validation para series temporales.
    Respeta el orden temporal: cada fold usa datos pasados para entrenar
    y datos futuros para validar.
    """
    def __init__(self, n_splits=5):
        self.n_splits = n_splits
    
    def split(self, X):
        n_samples = len(X)
        fold_size = n_samples // (self.n_splits + 1)
        
        for i in range(self.n_splits):
            # Train: desde el inicio hasta el punto de corte
            train_end = fold_size * (i + 2)
            train_indices = np.arange(0, train_end - fold_size)
            
            # Validation: el fold siguiente
            val_start = train_end - fold_size
            val_end = train_end
            val_indices = np.arange(val_start, val_end)
            
            yield train_indices, val_indices

print("✓ TimeSeriesKFold definido")


In [ ]:
# INSTRUCCIÓN: Ejecutar para definir la función que crea modelos

def crear_modelo(input_shape, arquitectura, learning_rate=0.001):
    """
    Crea un modelo de red neuronal con la arquitectura especificada.
    
    Args:
        input_shape: Número de features de entrada
        arquitectura: Lista con número de neuronas por capa, ej: [64, 32, 16]
        learning_rate: Tasa de aprendizaje del optimizador
        
    Returns:
        Modelo compilado
    """
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    
    # Agregar capas ocultas
    for neurons in arquitectura:
        model.add(layers.Dense(neurons, activation='relu'))
    
    # Capa de salida
    model.add(layers.Dense(1))
    
    # Compilar
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

print("✓ Función crear_modelo() definida")


In [ ]:
# INSTRUCCIÓN: Ejecutar para definir qué hiperparámetros probar
# PUEDES MODIFICAR ESTOS VALORES según lo que quieras probar

# Hiperparámetros a probar
HYPERPARAMETERS = {
    'window_size': [3, 5, 7],  # Tamaños de ventana
    'arquitectura': [
        [64, 32, 16],      # Red profunda
        [128, 64],         # Red ancha
        [32, 32, 32],      # Red uniforme
        [64, 32],          # Red simple
    ],
    'learning_rate': [0.001, 0.0001],  # Tasas de aprendizaje
}

# Configuración del experimento
N_SPLITS = 5           # Número de folds para K-Fold
EPOCHS = 100           # Épocas de entrenamiento
BATCH_SIZE = 32        # Tamaño del batch
VERBOSE = 0            # 0=silencioso, 1=verbose

print("=" * 80)
print("CONFIGURACIÓN DEL EXPERIMENTO")
print("=" * 80)
print(f"K-Fold splits: {N_SPLITS}")
print(f"Épocas por modelo: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print("\nHiperparámetros a validar:")
print(f"  - Window sizes: {HYPERPARAMETERS['window_size']}")
print(f"  - Arquitecturas: {len(HYPERPARAMETERS['arquitectura'])} variantes")
print(f"  - Learning rates: {HYPERPARAMETERS['learning_rate']}")

# Calcular total de configuraciones
total_configs = (len(HYPERPARAMETERS['window_size']) * 
                 len(HYPERPARAMETERS['arquitectura']) * 
                 len(HYPERPARAMETERS['learning_rate']))
total_entrenamientos = total_configs * N_SPLITS

print(f"\nTotal de configuraciones: {total_configs}")
print(f"Total de entrenamientos: {total_entrenamientos}")
print(f"⏱️ Tiempo estimado: ~{total_entrenamientos * 2 // 60} minutos")
print("=" * 80)

In [ ]:
# INSTRUCCIÓN: ⚠️ ESTA ES LA CELDA PRINCIPAL - Puede tardar varios minutos
# Esta celda ejecuta todo el proceso de validación cruzada

print("\n" + "=" * 80)
print("INICIANDO K-FOLD CROSS-VALIDATION")
print("=" * 80)

resultados = []
config_num = 0

# Iterar sobre todas las combinaciones de hiperparámetros
for window_size, arquitectura, lr in product(
    HYPERPARAMETERS['window_size'],
    HYPERPARAMETERS['arquitectura'],
    HYPERPARAMETERS['learning_rate']
):
    config_num += 1
    
    print(f"\n{'=' * 80}")
    print(f"CONFIGURACIÓN {config_num}/{total_configs}")
    print(f"{'=' * 80}")
    print(f"  Window size: {window_size}")
    print(f"  Arquitectura: {arquitectura}")
    print(f"  Learning rate: {lr}")
    
    # Crear ventanas con este window_size usando SOLO train+val data
    X, y = crear_ventanas(df_train_val, features, window_size=window_size)
    
    # Inicializar K-Fold
    kfold = TimeSeriesKFold(n_splits=N_SPLITS)
    
    # Almacenar scores de cada fold
    fold_scores = []
    
    # Iterar sobre cada fold
    for fold_num, (train_idx, val_idx) in enumerate(kfold.split(X), 1):
        print(f"  → Fold {fold_num}/{N_SPLITS}... ", end='')
        
        # Separar train y validation para este fold
        X_train_fold = X[train_idx]
        y_train_fold = y[train_idx]
        X_val_fold = X[val_idx]
        y_val_fold = y[val_idx]
        
        # Escalar datos (IMPORTANTE: fit solo en train)
        scaler_X_fold = MinMaxScaler()
        scaler_y_fold = MinMaxScaler()
        
        X_train_scaled = scaler_X_fold.fit_transform(X_train_fold)
        X_val_scaled = scaler_X_fold.transform(X_val_fold)
        
        y_train_scaled = scaler_y_fold.fit_transform(y_train_fold.reshape(-1, 1)).flatten()
        y_val_scaled = scaler_y_fold.transform(y_val_fold.reshape(-1, 1)).flatten()
        
        # Crear modelo
        tf.random.set_seed(SEED + fold_num)
        model = crear_modelo(X_train_scaled.shape[1], arquitectura, lr)
        
        # Entrenar
        history = model.fit(
            X_train_scaled, y_train_scaled,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            validation_data=(X_val_scaled, y_val_scaled),
            verbose=VERBOSE
        )
        
        # Evaluar en validation
        y_pred_scaled = model.predict(X_val_scaled, verbose=0)
        y_pred = scaler_y_fold.inverse_transform(y_pred_scaled)
        y_val_real = scaler_y_fold.inverse_transform(y_val_fold.reshape(-1, 1))
        
        # Calcular métricas
        mae = mean_absolute_error(y_val_real, y_pred)
        rmse = np.sqrt(mean_squared_error(y_val_real, y_pred))
        
        fold_scores.append({
            'mae': mae,
            'rmse': rmse
        })
        
        print(f"MAE: {mae:.4f}m")
    
    # Calcular promedios de los folds
    mae_mean = np.mean([s['mae'] for s in fold_scores])
    mae_std = np.std([s['mae'] for s in fold_scores])
    rmse_mean = np.mean([s['rmse'] for s in fold_scores])
    rmse_std = np.std([s['rmse'] for s in fold_scores])
    
    # Guardar resultados de esta configuración
    resultados.append({
        'window_size': window_size,
        'arquitectura': str(arquitectura),
        'learning_rate': lr,
        'mae_mean': mae_mean,
        'mae_std': mae_std,
        'rmse_mean': rmse_mean,
        'rmse_std': rmse_std,
        'mae_folds': [s['mae'] for s in fold_scores],
        'rmse_folds': [s['rmse'] for s in fold_scores]
    })
    
    print(f"\n  📊 Resultado promedio ({N_SPLITS} folds):")
    print(f"     MAE: {mae_mean:.4f} ± {mae_std:.4f} m")
    print(f"     RMSE: {rmse_mean:.4f} ± {rmse_std:.4f} m")

print("\n" + "=" * 80)
print("✓ VALIDACIÓN CRUZADA COMPLETADA")
print("=" * 80)


In [ ]:
df_resultados = pd.DataFrame(resultados)

# Ordenar por MAE (menor es mejor)
df_resultados_sorted = df_resultados.sort_values('mae_mean')

print("\n" + "=" * 80)
print("TOP 5 MEJORES CONFIGURACIONES (según MAE)")
print("=" * 80)
print("\nWindow | Arquitectura    | LR     | MAE (m)         | RMSE (m)")
print("-" * 80)

for idx, row in df_resultados_sorted.head(5).iterrows():
    print(f"{row['window_size']:^6} | {row['arquitectura']:^15} | {row['learning_rate']:.4f} | "
          f"{row['mae_mean']:.4f}±{row['mae_std']:.4f} | "
          f"{row['rmse_mean']:.4f}±{row['rmse_std']:.4f}")

# Mejor configuración
mejor_config = df_resultados_sorted.iloc[0]

print("\n" + "=" * 80)
print("🏆 MEJOR CONFIGURACIÓN ENCONTRADA")
print("=" * 80)
print(f"  Window size: {mejor_config['window_size']}")
print(f"  Arquitectura: {mejor_config['arquitectura']}")
print(f"  Learning rate: {mejor_config['learning_rate']}")
print(f"  MAE: {mejor_config['mae_mean']:.4f} ± {mejor_config['mae_std']:.4f} m")
print(f"  RMSE: {mejor_config['rmse_mean']:.4f} ± {mejor_config['rmse_std']:.4f} m")
print("=" * 80)



In [ ]:
# Parámetros de entrenamiento
EPOCHS = 100        # número de veces que recorre todo el dataset (ajustá según rendimiento)
BATCH_SIZE = 32     # tamaño de los lotes de entrenamiento

print(f"Configuración de entrenamiento -> EPOCHS: {EPOCHS}, BATCH_SIZE: {BATCH_SIZE}")


In [ ]:
print("\n" + "=" * 80)
print("ENTRENANDO MODELO FINAL CON MEJORES HIPERPARÁMETROS")
print("=" * 80)

# Extraer mejores hiperparámetros
best_window = int(mejor_config['window_size'])
best_arquitectura = eval(mejor_config['arquitectura'])
best_lr = (mejor_config['learning_rate'])

print(f"\nUsando configuración óptima:")
print(f"  Window size: {best_window}")
print(f"  Arquitectura: {best_arquitectura}")
print(f"  Learning rate: {best_lr}")

# Crear ventanas con TODO el train+val set (ahora pasando 'features')
X_train_final, y_train_final = crear_ventanas(df_train_val, features, window_size=best_window)

# Escalar datos
scaler_X_final = MinMaxScaler()
scaler_y_final = MinMaxScaler()

X_train_final_scaled = scaler_X_final.fit_transform(X_train_final)
y_train_final_scaled = scaler_y_final.fit_transform(y_train_final.reshape(-1, 1)).flatten()

# ✅ Guardar los scalers para usar luego en predicción
import joblib
joblib.dump(scaler_X_final, "scaler_X_final.pkl")
joblib.dump(scaler_y_final, "scaler_y_final.pkl")

print("✓ Scalers guardados: 'scaler_X_final.pkl' y 'scaler_y_final.pkl'")

# Crear y entrenar modelo final
print("\n🔄 Entrenando modelo final...")
tf.random.set_seed(SEED)
modelo_final = crear_modelo(X_train_final_scaled.shape[1], best_arquitectura, best_lr)

history_final = modelo_final.fit(
    X_train_final_scaled, y_train_final_scaled,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.2,  # Solo para monitorear, no para seleccionar hiperparámetros
    verbose=1
)

print("\n✓ Modelo final entrenado exitosamente")

# Guardar modelo
modelo_final.save('modelo_final_optimizado.h5')
print("✓ Modelo guardado en 'modelo_final_optimizado.h5'")

In [ ]:
# Evaluación final mínima en test set (sólo MAE / RMSE) - versión compacta para iteraciones rápidas
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Crear ventanas para el test set (asegúrate que best_window y modelo/scalers existen)
X_test_final, y_test_final = crear_ventanas(df_test, features, window_size=best_window)

# Escalar y predecir (usando scalers guardados en entrenamiento final)
X_test_final_scaled = scaler_X_final.transform(X_test_final)
y_pred_test_scaled = modelo_final.predict(X_test_final_scaled, verbose=0)
y_pred_test = scaler_y_final.inverse_transform(y_pred_test_scaled)
y_test_real = y_test_final.reshape(-1, 1)

mae_test = mean_absolute_error(y_test_real, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test_real, y_pred_test))

print('\n📊 EVALUACIÓN MÍNIMA EN TEST SET:')
print(f'  MAE: {mae_test:.6f} m')
print(f'  RMSE: {rmse_test:.6f} m')


In [ ]:
# CELDA DE PREDICCIONES ITERATIVAS / GRÁFICOS: eliminado para acelerar iteraciones.
# Si necesitas volver a ejecutar análisis/plots, recupera esta funcionalidad desde el backup original.
print('Predicciones iterativas y visualizaciones eliminadas en esta copia para acelerar el pipeline.')